In [1]:
import torch as th
import torch.nn as nn
from torch.optim import Adam

import gymnasium as gym

from jarl.modules.mlp import MLP
from jarl.envs.gym import TorchGymEnv
from jarl.data.dict import DotDict
from jarl.data.buffer import LazyBuffer
from jarl.modules.operator import Critic
from jarl.modules.encoder import FlattenEncoder
from jarl.modules.policy import CategoricalPolicy

from jarl.train.optim import Optimizer
from jarl.train.update.critic import MSECriticUpdate
from jarl.train.update.policy import ClippedPolicyUpdate
from jarl.train.update.ppo import PPOUpdate
from jarl.train.graph import TrainGraph
from jarl.train.sample.base import BatchSampler

from jarl.train.modify.compute import (
    ComputeValues,
    ComputeLogProbs,
    ComputeAdvantages,
    ComputeReturns
)

In [2]:
env = gym.make('LunarLander-v2')
env = TorchGymEnv(env)

In [3]:
policy = CategoricalPolicy(
    head=FlattenEncoder(),
    body=MLP(func=nn.Tanh, dims=[64, 64])
).build(env)

critic = Critic(
    head=FlattenEncoder(), 
    body=MLP(func=nn.Tanh, dims=[64, 64]),
).build(env)

In [4]:
ppo_block = (
    TrainGraph(
        BatchSampler(64, num_epoch=10),
        PPOUpdate(
            2048, policy, critic, optimizer=Optimizer(Adam, lr=3e-4), ent_coef=0.01
        )
    )
    .add_modifier(ComputeAdvantages())
    .add_modifier(ComputeLogProbs(policy))
    .add_modifier(ComputeReturns())
    .add_modifier(ComputeValues(critic))
    .compile()
)

In [5]:
import numpy as np

def get_episodic_return(data):
    don = data.don
    rew = data.rew
    ret = []
    ep_ret = 0
    for i, d in enumerate(don):
        ep_ret += rew[i].item()
        if d:
            ret.append(ep_ret)
            ep_ret = 0
    return ret

In [76]:
policy(env.obs_space.sample(), sample=False)

tensor(3)

In [7]:
buffer = LazyBuffer(2048)

rews = []

obs = env.reset()
for t in range(int(1e6)):
    with th.no_grad():
        trs = DotDict(
            obs=obs, 
            act=policy(obs)
        )
    exp, obs = env.step(trs=trs)
    buffer.store(exp)

    if ppo_block.ready(t):
        data = buffer.serve()
        batch_info = ppo_block.update(data)
        rews.extend(get_episodic_return(data))
        batch_info.update()
        print(t, batch_info | dict(rew=np.mean(rews[-100:])))

2048 {'policy_loss': 0.0882226973772049, 'entropy_loss': -0.013603923842310905, 'approx_kl': -0.050149738788604736, 'critic_loss': 843.47998046875, 'rew': -167.9350174453787}
4096 {'policy_loss': 0.09005573391914368, 'entropy_loss': -0.013443437404930592, 'approx_kl': 0.029023338109254837, 'critic_loss': 559.9609375, 'rew': -168.3487526226917}
6144 {'policy_loss': 0.09022151678800583, 'entropy_loss': -0.013416316360235214, 'approx_kl': -0.018514204770326614, 'critic_loss': 770.3955078125, 'rew': -177.91880817954237}
8192 {'policy_loss': 0.07929959148168564, 'entropy_loss': -0.013540162704885006, 'approx_kl': -0.021574873477220535, 'critic_loss': 597.94482421875, 'rew': -169.05179533111377}
10240 {'policy_loss': 0.05577373504638672, 'entropy_loss': -0.013530528172850609, 'approx_kl': -0.016158761456608772, 'critic_loss': 395.8072509765625, 'rew': -167.91428247433504}
12288 {'policy_loss': 0.053293317556381226, 'entropy_loss': -0.013525242917239666, 'approx_kl': 0.014239989221096039, 'cr